## Imports

In [8]:
import pandas as pd
import numpy as np

import nltk

import re

import spacy

import matplotlib.pyplot as plt

#### Reading the File

In [9]:
df = pd.read_csv('clean_masks.csv')
df.head()

,natural_key,id,author_id,body,country,publishing_date,coordinates,place,user_country,retweet_count,...,name,favourites_count,friends_count,location,extracted_country,created_at,listed_count,project_tweet_count,rtt_author_ids,retweet_dates
0,1326183672739983362,703645d2-242b-11eb-9b3e-02420a000016,2995705253,Je suis allée faire quelques courses chez Carr...,None,2020-11-10 15:23:21,None,None,FR,3,...,(((Viviane Bchnn))) 🇫🇷 🇪🇺 🇮🇱,85758,1531,Marseille au bord de la mer,FR,2015-01-24 11:59:26,132,4,"[1073646674030018560, 2937667860]",['7944 2020-11-10 18:15:45\nName: retweet_d...
1,1326224675731550208,1717c796-242b-11eb-9b3e-02420a000016,495846843,Suis en colère ! Sans être une démarche scient...,None,2020-11-10 18:06:17,None,None,FR,2,...,A.Levy-Mozziconacci,4575,1073,"Marseille, PACA",FR,2012-02-18 11:13:11,71,2,"[882287298309763073, 10125142]",['3213 2020-11-10 18:29:35\nName: retweet_d...
2,1326417031441342465,5b9204f0-242a-11eb-9b3e-02420a000016,38634848,#Masques chirurgicaux : il est possible de les...,None,2020-11-11 06:50:38,None,None,FR,6,...,Dominique Lang,65467,6949,"Saint-Avold, France",FR,2009-05-08 09:11:16,443,1,"[745923049292201985, 797332709504937984, 38634...",['3215 2020-11-11 07:39:41\nName: retweet_d...
3,1326398902568955910,65704270-242a-11eb-9b3e-02420a000016,15339119,Le masque chirurgical est bel et bien lavable ...,None,2020-11-11 05:38:36,None,None,FR,51,...,Fred Softway,27391,5002,France paris,FR,2008-07-07 05:25:55,114,1,"[1257051098, 252070564, 947585901340905472, 92...",['1442 2020-11-11 05:41:07\nName: retweet_d...
4,1326434827420258304,47adc3c0-242a-11eb-9b3e-02420a000016,36383320,"Attestations, port du masque, circulation limi...",None,2020-11-11 08:01:21,None,None,FR,8,...,France Culture,2205,1806,93.5 à Parispp,FR,2009-04-29 15:23:52,5637,1,"[933354198, 1243597919925735429, 1279274995429...",['10679 2020-11-11 08:03:36\nName: retweet_...


## Preprocessing

In [10]:
def get_hyperlinks(text):
    ## finds all hyperlinks in the text
    return re.findall(r"http\S+", text)

def rmv_hyperlink(text):
    ## removes all hyperlinks of the text
    return re.sub(r"http\S+", "", text)

def get_hashtags(text):
    ## finds all hashtags of the text
    return re.findall(r"#\S+", text)

def rmv_hashtags(text):
    ## removes all hashtags of the text
    return re.sub(r"#\S+", "", text)

def get_users(text):
    return re.findall(r"@\S+", text)

def scrap_tweets(df):
    df['hyperlinks'] = df['body'].map(
        get_hyperlinks
    )
    df['Title'] = df['body'].map(
        rmv_hyperlink
    )
    df['hashtags'] = df['body'].map(
        get_hashtags
    )
    df['Title'] = df['body'].map(
        rmv_hashtags
    )
    df['referenced_users'] = df['body'].map(
        get_users
    )

In [11]:
def clean_tweets(df):
    res = df[df['user_country'] == 'FR']
    P = (len(df) - len(res)) / len(df)
    print('{}% of tweets were removed'.format(P))
    return res
    
only_FR_df = clean_tweets(df)

0.0% of tweets were removed


#### tokenizing in sentences and PoS

In [12]:
## french PoS:
## https://github.com/cmchurch/nltk_french/blob/master/french-nltk.py

## https://nlp.stanford.edu/software/CRF-NER.html

nlp = spacy.load('fr_core_news_sm')

only_FR_df['body'].map(
    nltk.sent_tokenize
)

0      [Je suis allée faire quelques courses chez Car...
1      [Suis en colère !, Sans être une démarche scie...
2      [#Masques chirurgicaux : il est possible de le...
3      [Le masque chirurgical est bel et bien lavable...
4      [Attestations, port du masque, circulation lim...
                             ...                        
302    [En mars tout le monde voulait des masques et ...
303    [Dans 15 jours fermeture des collèges ?, A cro...
304    [La faible mobilisation, mardi, masque, pour l...
305    [Absence de RIO et masques par timidité,  pas ...
306    [L’ESS, c’est être utile socialement Exemple d...
Name: body, Length: 307, dtype: object

### Creating vocabulary

#### Lemmatize

In [13]:
from spacy_lefff import LefffLemmatizer, POSTagger

pos = POSTagger()

french_lemmatizer = LefffLemmatizer(after_melt = True)

nlp.add_pipe(pos, name='pos', after='parser')
nlp.add_pipe(french_lemmatizer, name='lefff', after='pos')

2020-11-15 12:49:08,198 - spacy_lefff.downloader - INFO - data already set up
2020-11-15 12:49:08,198 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading lexicon...
2020-11-15 12:49:08,561 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading tags...
2020-11-15 12:49:08,580 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr...
2020-11-15 12:49:08,891 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr: done
2020-11-15 12:49:08,892 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-11-15 12:49:08,892 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-11-15 12:49:09,253 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [22]:
def lemmatize(text):
    return [doc.lemma_ for doc in nlp(text)]
    

lemmatized = only_FR_df['body'].map(
    lemmatize
)

2020-11-15 12:49:33,534 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,557 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,575 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,590 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,600 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,617 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,638 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,653 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,675 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,692 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,708 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:33,732 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-15 12:49:35,446 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,470 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,488 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,509 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,530 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,553 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,576 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,598 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,616 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,631 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,647 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:35,671 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-15 12:49:37,234 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,245 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,259 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,273 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,297 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,310 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,331 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,343 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,355 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,378 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,405 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:37,419 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-15 12:49:39,023 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,034 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,046 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,069 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,095 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,125 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,152 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,172 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,185 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,206 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,237 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-15 12:49:39,258 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

In [23]:
for l in lemmatized[:20]:
    print(l)

['je', 'être', 'aller', 'faire', 'quelque', 'course', 'chez', 'Carrefour', '.', 'en', 'arriver', 'à', 'le', 'caisse', 'je', 'me', 'apercevoir', 'que', 'je', 'avoir', 'oublier', 'de', 'mettre', 'mon', 'masqu', '.....', 'ni', 'le', 'vigil', 'à', 'le', 'entrée', 'ni', 'le', '5', 'ou', '6', 'salarier', 'de', 'magasin', 'ni', 'même', 'un', 'client', 'ne', 'me', "l'", 'avoir', 'faire', 'remarquer', '.']
['être', 'en', 'colère', '!', 'sans', 'être', 'un', 'démarche', 'scientifique', ',', 'je', 'me', 'rendre', 'compte', 'que', 'seulement', '30', 'pourcent', 'de', 'marseillais', 'porter', 'le', 'masque', 'correctement', 'et', 'cela', 'quelque', 'soit', 'le', 'quartier', 'de', '#', 'marseill', '!', 'ressaisissez', 'vous', 'svp', '!']
['#', 'masques', 'chirurgical', ':', 'il', 'être', 'possible', 'de', 'le', 'laver', 'et', 'de', 'le', 'réutiliser', '10', 'fois', ',', 'selon', '@UFC', '-', 'quechoisir', '🤔', '🤔', '🤔', 'https://t.co/pPMPcWXyBF']
['le', 'masque', 'chirurgical', 'être', 'bel', 'et', 

In [24]:
lemmatized = lemmatized.map(
    lambda x : " ".join(x)
)

#### Removing unwanted characters

In [25]:
def rmv_char(text):
    ## removes all unwanted characters of the text
    text = re.sub("""[\n´]""", " ", text)
    text = re.sub("""["'+"*%&/()"=¦@#°§¬|¢\[\]\-\_\—‘’“”•`\^{}~¥©?.,:!$;«»<>]""", "", text)
    return re.sub("""\d""", " ", text)

In [26]:
filtered = lemmatized.map(
    lambda r: r.lower()
).map(
    rmv_char
)
filtered

0      je être aller faire quelque course chez carref...
1      être en colère  sans être un démarche scientif...
2       masques chirurgical  il être possible de le l...
3      le masque chirurgical être bel et bien lavable...
4      attestation  port de masque  circulation limit...
                             ...                        
302    en mars tout le monde vouloir de masque et un ...
303    dans    jour fermeture de collège  avoir croir...
304    le faible mobilisation  mardi  masqu  pour le ...
305    absence de rio et masque par timidité    pas p...
306    l ess  c être être utile socialement exemple d...
Name: body, Length: 307, dtype: object

#### Tokenizing and Removing stopwords

In [27]:
from nltk.corpus import stopwords

stopwords = stopwords.words('french')

def rmv_stop_word(tokens):
    return [x for x in tokens if x not in stopwords]

def filter_characters(tokens):
    return [x for x in tokens if x.isalpha()]

tokens = filtered.map(
    nltk.word_tokenize
).map(
    rmv_stop_word
).map(
    filter_characters
)

tokens

0      [être, aller, faire, quelque, course, chez, ca...
1      [être, colère, sans, être, démarche, scientifi...
2      [masques, chirurgical, être, possible, laver, ...
3      [masque, chirurgical, être, bel, bien, lavable...
4      [attestation, port, masque, circulation, limit...
                             ...                        
302    [mars, tout, monde, vouloir, masque, vaccin, a...
303    [jour, fermeture, collège, avoir, croire, ça, ...
304    [faible, mobilisation, mardi, masqu, syndicat,...
305    [absence, rio, masque, timidité, parce, avoir,...
306    [ess, être, être, utile, socialement, exemple,...
Name: body, Length: 307, dtype: object

## Trying out gensim

In [31]:
from gensim.models import Word2Vec
model = Word2Vec(tokens, min_count = 1)

2020-11-11 14:35:21,872 - gensim.models.word2vec - INFO - collecting all words and their counts
2020-11-11 14:35:21,872 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-11 14:35:21,913 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 183459 words, keeping 29751 word types
2020-11-11 14:35:21,956 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 357153 words, keeping 50145 word types
2020-11-11 14:35:21,985 - gensim.models.word2vec - INFO - collected 62733 word types from a corpus of 485281 raw words and 27314 sentences
2020-11-11 14:35:21,986 - gensim.models.word2vec - INFO - Loading a fresh vocabulary
2020-11-11 14:35:22,058 - gensim.models.word2vec - INFO - effective_min_count=1 retains 62733 unique words (100% of original 62733, drops 0)
2020-11-11 14:35:22,058 - gensim.models.word2vec - INFO - effective_min_count=1 leaves 485281 word corpus (100% of original 485281, drops 0)


In [32]:
for w in np.sort(list(model.wv.vocab.keys())):
    print(w)

a
aa
aaa
aaaaaaaahhhhh
aaaah
aaah
aaamiiraaa
aaavocat
aaccliv
aacuh
aafrench
aah
aahoult
aahyim
aajfma
aajjaax
aajxq
aamaryilis
aamquee
aan
aanwww
aaon
aaoqiz
aap
aapresidglobalfarmernet
aarbitre
aarjmjuof
aaron
aaronderfel
aarongr
aaronjonesraven
aaronvells
aatea
aau
aax
aaxrfyev
aazzazzeell
ab
aba
abacan
abadal
abamako
abandon
abandonne
abandonner
abandonnera
abarader
abarleduc
abaslesmasque
abasourdi
abasourdie
abasourdir
abat
abattage
abattoir
abattre
abattu
abb
abba
abbad
abbas
abbasside
abbatage
abbater
abbeamar
abbegrosjean
abbeyhartmann
abbicyclettew
abbotasher
abbott
abbythelast
abc
abcarto
abcxc
abd
abdalaye
abde
abdel
abdelcheref
abdeldjalil
abdeldz
abdelenvrai
abdelenvrer
abdelkarim
abdelkrimdehimi
abdellakhal
abdellalihajjat
abdellatif
abdelmalik
abdelmehdi
abdelmrc
abdelzedz
abdia
abdlim
abdoul
abdoulayek
abdoulkamkam
abdoushc
abdulelrazad
abdulquira
abdusselamzahmer
abdxcym
abec
abeille
abelamundala
abenayvanna
aberotusiruk
aberrant
aberration
aberut
abgah
abhgmjqq
abidj

amidon
amie
amiens
amies
amiinamarante
amikriou
amilhat
amimarsupial
aminata
amincissante
amine
amineeeblk
aminejendnba
aminek
aminelabrech
aminematu
aminematue
aminevilani
aminfumi
amins
aminsx
amiot
amiral
amirami
amirkhadir
amis
amisdelaterre
amish
amissamya
amissi
amisumr
amitie
amitier
amitié
amivert
amk
amkvf
aml
amlil
amm
ammartns
ammljpqyj
amnddu
amnesty
amnestyfrance
amnestyhk
amo
amocz
amoin
amoindrie
amoindrissement
amondechene
among
amont
amorce
amorcer
amorouayechemo
amortir
amorçage
amour
amouracamille
amoureusement
amoureux
amovible
amp
ampbilel
amphechantenay
amphi
amphibie
amplement
ampleur
amplification
amplifier
amputer
ampélographiqu
amqrkl
amsskho
amsterdam
amsterkham
amtarox
amu
amufzfb
amugnier
amuh
amundifr
amus
amusant
amusauchocolat
amuse
amusent
amuser
amuseur
amvcle
amvcler
amvej
amvz
amwalvin
amxnataee
amyaevc
amylawrence
amypeng
amyway
amène
amélie
amélioration
améliore
améliorentc
améliorer
amélioré
aménagement
aménagemer
aménager
américain
américaine
amé

bannissant
bannissement
bannièr
bannière
bannon
banqu
banque
banquemondial
banquet
banquier
banquise
bantou
bantous
bantu
banuhmomo
banzaï
bao
baobab
baolong
baomix
baoo
baozi
bapan
bapoecu
baptiser
baptistemmt
baptisteteddy
baptisteth
baptistperrault
baptisé
bar
barackcalva
barackobama
baragouine
baraka
barakacity
barakatuh
barakzibith
baraque
baraquement
baratin
baratine
baratineur
barbara
barbarapayotte
barbarapompili
barbare
barbarelou
barbarie
barbatruc
barbe
barbecue
barbenoirrr
barberis
barberousse
barbeuzfr
barbiana
barbieux
barbikubrick
barboe
barbotte
barbouze
barbraig
barbu
barbès
barca
barcelone
bard
bardamuacab
barder
bardock
bardockamazighi
bardouville
bargxh
bari
baric
barisanjujur
barjo
barkettedelu
barkhane
barks
barnb
barnum
baron
baroudlkhayl
barquette
barr
barrage
barragebouygue
barrais
barrakiaxonn
barralsenior
barre
barrer
barriamorgan
barrièr
barrière
barriére
barré
bars
bart
barthes
barthruzza
barthès
barthélémy
bartjetweet
bartlebybye
bartlet
bartomeu
barwell
b

boycotton
boycottproduitschinois
boycotts
boycottzz
boyer
boykaa
boyquam
boys
boyslovefrance
boystory
boz
bozclayj
bozokra
boîte
boîtedesauvegarde
boîtier
bp
bpbfa
bpbxymisk
bpci
bpcoom
bpctr
bpemxfr
bpf
bpfywmerci
bpi
bpifranc
bpifrance
bpl
bpmnfp
bpolitic
bpompili
bprycqc
bpshu
bptnd
bpze
bq
bqad
bqc
bqcvysar
bqe
bqeroz
bqj
bqjkt
bqjplsu
bqkux
bqrm
bqtaq
bquestel
bquzg
bqvd
bqvt
bqxrelko
br
bra
brabet
bracelet
bracket
braconnage
braconnier
brad
bradbelek
braderie
brahim
brahmapoutre
brahmsachelhi
brahosdias
braid
brainelalleud
brainois
brainstim
brainz
brair
brais
braise
braisé
bramavie
brambiye
bramsgue
bramsi
branaajean
branche
brancher
branchouillerie
branchu
branché
brand
brander
brandir
brandonlks
brankomilan
branle
branler
branlette
branlettecolie
branleur
branquignol
branquignols
braq
braquet
bras
brasegalijuli
brasier
brasil
brassage
brassant
brasser
brassicacée
brassicanigra
bratislava
braudel
brauliojcz
braun
brave
bravenewchina
bravitude
bravo
bravos
braxoul
brazza
brazzav

claradumez
clarapergola
clarence
clarifier
clarin
clarté
clash
clasher
clashtinder
classaction
classe
classement
classer
classeur
classification
classifier
classique
classé
classée
clattenburg
claude
claudeambomo
claudecamille
claudecologne
claudedanthony
claudedj
claudeelkhal
claudemery
claudemoniquet
claudeposternak
clauder
claudesteffen
claudetoken
claudetower
clause
claustration
clavicul
clavier
clavieraverti
claviesophie
clc
cldoooo
cle
clean
cleaner
cleans
clear
clearchannelfr
clearfield
clecourtier
clef
clefs
cleliemathia
clelievg
clemardele
clemautain
clemenceau
clemencebnrn
clemenceji
clemencetstr
clementbt
clementfabre
clementgdre
clementgrv
clementlanot
clementn
clementuchiha
clemfcr
clemjess
clemmauclair
clemstreetcity
cleopvtrvvll
clep
cler
clerc
clergeay
clergé
clermont
clermontferrand
cleum
clevy
clf
clgmalrauxcsm
clhebdo
clhoroquine
clic
clicher
clichy
cliché
clicli
clics
client
clientèle
clientéle
clignez
cliic
climat
climateaction
climatechange
climatefr
climatique
cl

...

tendance
tendancerouge
tendances
tendancieux
tendent
tendooo
tendraient
tendre
tendu
tenet
teneur
tenez
tenfishy
tengchong
tengger
tengjia
tenir
tennesseem
tennis
tennishallofame
tenon
tenser
tensibai
tension
tensions
tent
tentation
tentativ
tentative
tente
tentent
tenter
tentez
tention
tenu
tenue
tenvois
teo
tepco
ter
tera
teram
terana
teraofr
terence
terencechu
terencedesclain
terenceleungsf
terencestrick
tergiverser
term
terme
terminal
terminale
terminator
termine
termineeeee
terminer
terminologie
terminus
terminé
termite
termitière
terne
terr
terracid
terraformation
terrain
terrasse
terrasser
terre
terreatom
terreau
terredasie
terreest
terreetpeuple
terreetpeuples
terrell
terrencestr
terrenetfr
terrerares
terres
terresrare
terrestre
terrestrer
terreterre
terreur
terrible
terriblement
terrien
terrienne
terrier
terrifiant
terrifiante
terrifier
territoir
territoire
territoires
territoiresdesavoirs
territorial
territoriale
territorialement
territorialité
terro
terroir
terrorise
terrori

tours
tous
tousart
touscontremacron
touscontremacronjour
tousensembl
toussa
toussaint
tousse
toussecnum
toussent
tousser
tousseur
tousssssaaaaaa
tout
toutalimentation
toute
toutefois
touteleurope
toutenkamion
toutes
toutl
toutogm
toutou
toutpourlamuzik
toutpuissant
touttrump
touuuut
touuuute
toux
tovarichlama
towersight
towintou
toxic
toxiciter
toxicité
toxico
toxine
toxique
toxiquegratuit
toxiquer
toxylizumab
toyette
toz
tozo
tp
tpasmnkho
tpd
tpdach
tpdxtbnc
tpdzu
tpe
tpei
tpi
tpijot
tpk
tplpndp
tpmnkgvkx
tpmp
tpmpahck
tpnju
tpo
tpoexx
tpq
tprincedelamour
tps
tpujour
tpwe
tpwhqeke
tpwx
tpx
tqa
tqdm
tqe
tqecfwq
tql
tqlfpod
tqmt
tqnhe
tqoahsj
tqpf
tqqowfhu
tqs
tqt
tqvbusl
tr
trabailler
trabouye
tracabilite
tracage
tracasser
trace
tracent
tracer
traces
traceur
traceybthierry
tracing
track
tracker
tracking
tract
tractation
tracte
tracteur
traction
tracé
trad
trade
tradefutur
traderform
tradi
tradifourchette
trading
tradition
traditionelle
traditionfr
traditionnal
traditionnalisme
traditio

uruguayen
urugwirovillage
urumqi
uruy
urwu
urx
urz
us
usa
usachine
usadepuis
usaenfrancais
usaeu
usage
usager
usages
usain
usainbolt
usaindeaustralietaiwanjapon
usambdrc
usason
usb
usbeketrica
uscqi
usd
usda
usdcny
use
used
uselessmeteor
usen
user
usherbrooke
ushiwa
ushtrija
usin
usine
usinedigital
usinenouvelle
usiner
using
usjliban
usm
usmca
usmyxn
usnavy
usoytn
usp
uspix
uspqgyxg
usrm
uss
ussss
ustellar
usual
usuelle
usuldufutur
usurpateur
usvdu
usw
usxh
usy
ut
uta
utalyon
utbgyib
utbufnyyz
utdf
utero
utf
utile
utilis
utilisable
utilisateur
utilisation
utilise
utilisent
utiliser
utilitariste
utilité
utjeanjaure
utk
utmc
utmqlv
utn
utoirt
utopie
utopique
utopiste
utsuls
utt
uttj
uttroyes
utuxem
utygge
utérus
uu
uub
uud
uudg
uujxkf
uukbyi
uunxkg
uuoleug
uurto
uuvxjtv
uux
uuy
uuzrj
uv
uvbkt
uvcnn
uve
uvi
uvic
uvl
uvm
uvntsxvmcw
uvo
uvsqresearch
uvt
uvvltkem
uvwqu
uvwx
uvyd
uvzk
uvzqeaytw
uw
uwgs
uwhhxf
uwjuq
uwk
uwka
uwm
uwtca
uwvlijp
uwvqelno
uww
ux
uxaj
uxfdvud
uxh
uxlqq
uxpc
uxtbybk

whoacademy
whoisbrianlol
whoisekid
whoopi
whoracioa
whp
whqhqlrt
whqsdbcd
wht
whuoqf
whwkar
why
whyalwaysmanu
whydoc
whzxoi
wi
wibtxbta
wicca
widadebouthim
widadk
widely
widow
wiedenkennedy
wifi
wifyc
wig
wigc
wighx
wih
wiir
wiki
wikipedia
wikipédia
wiko
wil
wild
wildcard
wilfridcoby
wilfriedtabe
will
willdepam
william
williamrar
williamreymond
willoudu
willounet
willow
willowwisp
willyhuncho
wilohiei
wilson
wilwarin
win
winamaxsport
winatlab
windows
windson
windsor
windzesport
wine
wing
wingloong
wingofnight
winnie
winnipeg
wionew
wiqat
wir
wireles
wiri
wirok
wisethugtv
wish
wishlist
wisibility
wisionv
wissembarki
wissner
wistar
wistily
wisziv
witch
witcharle
witchknits
with
withjude
without
witness
witsel
wittersheim
wittmann
wiv
wix
wixafhyci
wixpuum
wizored
wizsl
wj
wjcozgshz
wjcrv
wjhynkbpd
wjlm
wjpykur
wjw
wjx
wjxp
wjyh
wjzgspa
wk
wkd
wkk
wkl
wklyxoa
wklzkctwo
wkn
wkqfd
wkqsb
wksllegy
wktnlr
wl
wla
wlad
wladtepes
wlb
wldnhd
wlfcmv
wlgat
wlgr
wlh
wlhkecmuss
wlidou
wlk
wll
wlle
wlm

émission
émissionsco
émittrice
émoticône
émotif
émotion
émotionnel
émouvant
émouvante
émouvoir
ému
énarque
énemi
énemis
énergie
énergiedupassé
énergique
énergivore
énergétique
énerv
énervait
énerve
énerver
énervé
énigme
énième
énoncer
énorme
énormes
énormité
énormément
énumèrer
énumérer
éolien
éolienne
éon
épais
épaisse
épaisseur
épancher
épanoui
épanouir
épanouissant
épargne
épargner
épars
éparses
épaul
épaule
éperdument
épervier
éphémère
épi
épice
épicentr
épicentre
épicer
épicéa
épicéas
épidemie
épidemiologiqu
épidémi
épidémie
épidémiologie
épidémiologique
épidémiologiquemer
épidémiologist
épidémiologiste
épidémique
épie
épier
épigenetique
épigrammes
épigénétique
épilepsie
épine
épineux
épingle
épingler
épiphanie
épiphénomène
épique
épisod
épisode
épissétou
épistémologie
épistémologique
épizootie
éplucher
éponge
éponger
époper
épopé
épopée
époqu
époque
épouse
épouser
époustouflant
époustoufler
épouvantable
épouvantail
époux
éprendre
épreuve
éprouve
éprouvette
épuisement
épuiser
épur

In [36]:
model.wv.most_similar(positive = ['trump'], negative = [], topn = 5)

[('meurt', 0.987456202507019),
 ('rapport', 0.9822913408279419),
 ('survivre', 0.9815309047698975),
 ('économie', 0.9801642894744873),
 ('perdre', 0.978381872177124)]

In [41]:
from nltk.cluster import KMeansClusterer

NUM_CLUSTERS = 21

kclusterer = KMeansClusterer(
    NUM_CLUSTERS,
    distance = nltk.cluster.util.cosine_distance,
    repeats = 3
)
assigned_clusters = kclusterer.cluster(
    model[model.wv.vocab],
    assign_clusters = True
)

c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [42]:
V = np.array(list(model.wv.vocab.keys()))
K = np.array(assigned_clusters)

In [43]:
for i in range(NUM_CLUSTERS):
    print(V[K == i])

['mediavenir' 'recoloniser' 'ci' ... 'regulation' 'joestj' 'droiteil']
['ça' 'passer' 'chinois' ... 'foudroyant' 'httpstcodwygqoaexh'
 'virtuellement']
['raid' 'ultra' 'surpasser' ... 'maso' 'incompéter' 'httpstcofrxappfmnq']
['possiblement' 'dommage' 'rapide' ... 'merveilleusement' 'carriere' 'kvy']
['chine' 'merovingien' 'chrétien' ... 'httpstcowtcwupvru' 'originellement'
 'musulmanie']
['ouïgours' 'region' 'asie' ... 'uunxkg' 'cureathee' 'poltron']
['pardonne' 'charlychin' 'iqhzg' ... 'httpstcouzp' 'xuaneszv' 'terrerares']
['quartier' 'thème' 'stv' ... 'rbi' '𝗛' 'devdetp']
['u' 'xnxbipx' 'kipithrill' ... 'shawn' 'fareshas' 'alyx']
['mort' 'wuhan' 'covid' ... 'boummm' 'bécasse' 'httpstcolrvioruikc']
['génétique' 'chercheur' 'voeu' ... 'jeangalano' 'spécifiquement'
 'expliquerait']
['httpstco' 'frappe' 'ptdrrrr' ... 'danielb' 'zdedededex' 'bruceduvic']
['gilford' 'information' 'disposer' ... 'jimcameron' 'clandestino' 'aisha']
['h' 'extraodinair' 'nuit' ... 'cuve' 'maure' 'departement

## Keras Pipeline

In [30]:
import tensorflow
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tokens = tokens.map(
    lambda l : ' '.join(l)
)

In [32]:
training_data = tokens.to_numpy(dtype = str)

In [33]:
## Note: can add ngrams if needed
vectorizer = TextVectorization(output_mode = 'int')
## will learn vocab
vectorizer.adapt(training_data)
integer_data = vectorizer(training_data)

print(integer_data[0])

tf.Tensor(
[   3   12    8  120  245   43 2334  382 2346  707    4  312   28    6
  215  836  622  215 1042  545  215  668    4    8 1118    0    0    0
    0    0    0    0    0    0    0], shape=(35,), dtype=int64)


In [34]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'masque',
 'être',
 'avoir',
 'httpstco',
 'masqu',
 'sans',
 'faire',
 'chirurgical',
 'covid',
 'port',
 'aller',
 'pouvoir',
 'fois',
 'plus',
 'tout',
 'cela',
 'dire',
 'laver',
 'selon',
 'non',
 'enfant',
 'ufc',
 'quand',
 'porter',
 'choisir',
 'réutilisable',
 'mettre',
 'lavable',
 'voir',
 'bien',
 'autre',
 'an',
 'réutiliser',
 'celui',
 'vaccin',
 'geste',
 'devoir',
 'bon',
 'vouloir',
 'falloir',
 'comme',
 'chez',
 'école',
 'via',
 'possible',
 'passer',
 'gens',
 'étude',
 'sanitaire',
 'quoi',
 'où',
 'mois',
 'masquer',
 'confinement',
 'après',
 'si',
 'policier',
 'novembre',
 'mort',
 'jour',
 'grand',
 'france',
 'rien',
 'public',
 'peu',
 'parent',
 'oui',
 'monde',
 'fait',
 'degré',
 'obligatoire',
 'lavage',
 'h',
 'famille',
 'contre',
 'virus',
 'venir',
 'sous',
 'r',
 'puis',
 'nouveau',
 'nez',
 'mesure',
 'k',
 'car',
 'barrière',
 'alors',
 'v',
 'servir',
 'rendre',
 'raison',
 'premier',
 'parce',
 'merci',
 'main',
 'gouvernement

In [68]:
vectorizer(['fois', 'port']).numpy().ravel()

array([14, 11], dtype=int64)

In [35]:
vocab_size = len(vectorizer.get_vocabulary())
vocab_size

2575

In [36]:
vocab = vectorizer.get_vocabulary()
reverse_mapping = {}
for i in range(vocab_size):
    reverse_mapping[i] = vocab[i]

In [37]:
sampling_table = tensorflow.keras.preprocessing.sequence.make_sampling_table(vocab_size)
# Should add some 0 padding such that the largest tweet doesn't get skipgramed with the next
(data_tuples, labels) = tensorflow.keras.preprocessing.sequence.skipgrams(
    integer_data.numpy().ravel(),
    vocab_size,
    window_size = 2,
    shuffle = True,
    sampling_table = sampling_table
)

# data_tuples = np.array(data_tuples)
# labels = np.array(labels)

In [38]:
d = [[int(x[0]), int(x[1])] for x in data_tuples]
labels = np.array([int(x) for x in labels])

target_words = np.array(d)[:,0]
context_words = np.array(d)[:,1]

In [62]:
len(labels)

6996

In [63]:
len(target_words)

6996

In [46]:
M = np.sum(sampling_table)
np.random.choice(vocab, 5, replace = False, p = sampling_table / M)

array(['enseignant', 'rhône', 'solde', 'attia', 'identité'], dtype='<U25')

## Model
### Implementation

In [51]:
embed_dim = 300

In [52]:
in_1 = tensorflow.keras.Input(1)
in_2 = tensorflow.keras.Input(1)


E = layers.Embedding(
    vocab_size,
    embed_dim,
    input_length = 1,
    name = 'embeddings'
)

E_1 = E(in_1)
E_2 = E(in_2)


D = tensorflow.keras.backend.sum(E_1 * E_2, axis = -1)

S = layers.Activation(activation = 'sigmoid')(D)

In [53]:
model = tensorflow.keras.Model(
    inputs = [in_1, in_2],
    outputs = [S]
)

model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1, 300)       772500      input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 1, 300)]     0           embeddings[0][0]      

### Validation callback

In [54]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'masque',
 'être',
 'avoir',
 'httpstco',
 'masqu',
 'sans',
 'faire',
 'chirurgical',
 'covid',
 'port',
 'aller',
 'pouvoir',
 'fois',
 'plus',
 'tout',
 'cela',
 'dire',
 'laver',
 'selon',
 'non',
 'enfant',
 'ufc',
 'quand',
 'porter',
 'choisir',
 'réutilisable',
 'mettre',
 'lavable',
 'voir',
 'bien',
 'autre',
 'an',
 'réutiliser',
 'celui',
 'vaccin',
 'geste',
 'devoir',
 'bon',
 'vouloir',
 'falloir',
 'comme',
 'chez',
 'école',
 'via',
 'possible',
 'passer',
 'gens',
 'étude',
 'sanitaire',
 'quoi',
 'où',
 'mois',
 'masquer',
 'confinement',
 'après',
 'si',
 'policier',
 'novembre',
 'mort',
 'jour',
 'grand',
 'france',
 'rien',
 'public',
 'peu',
 'parent',
 'oui',
 'monde',
 'fait',
 'degré',
 'obligatoire',
 'lavage',
 'h',
 'famille',
 'contre',
 'virus',
 'venir',
 'sous',
 'r',
 'puis',
 'nouveau',
 'nez',
 'mesure',
 'k',
 'car',
 'barrière',
 'alors',
 'v',
 'servir',
 'rendre',
 'raison',
 'premier',
 'parce',
 'merci',
 'main',
 'gouvernement

In [56]:
validation_set = ['président', 'positif','hydroxychloroquine']
validation_set = [vectorizer.get_vocabulary().index(w) for w in validation_set]
validation_set

[300, 302, 573]

In [57]:
num_words = 7

class ValidationCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        M = model.get_layer('embeddings').weights[0].numpy()
        M = M / np.linalg.norm(M, axis = 1)[:, np.newaxis]
        val_vects = M[validation_set]
        
        closest = np.argsort(np.cos(M @ val_vects.T), axis = 0)
        
        top = closest[:num_words, :]
        for (i,e) in enumerate(validation_set):
            print('word: {}'.format(reverse_mapping[e]))
            print('closest words: ')
            for c in top[:,i]:
                print('--')
                print(reverse_mapping[c])
            print('----------------')

In [58]:
model.fit(
    [target_words, context_words],
    labels,
    batch_size = 64,
    epochs = 30,
    callbacks = [ValidationCallback()],
    shuffle = True
)

Epoch 1/30
108/110 [============================>.] - ETA: 0s - loss: 0.6921word: président
closest words: 
--
président
--
rencontre
--
entre
--
confirme
--
regarder
--
artefr
--
effectuer
----------------
word: positif
closest words: 
--
positif
--
chose
--
cousin
--
également
--
accoucher
--
flic
--
vwwco
----------------
word: hydroxychloroquine
closest words: 
--
hydroxychloroquine
--
raoult
--
pgqmebj
--
mobiliser
--
masquesobligatoire
--
ph
--
employeur
----------------
110/110 [==============================] - 0s 4ms/step - loss: 0.6920
Epoch 2/30
110/110 [==============================] - ETA: 0s - loss: 0.6689word: président
closest words: 
--
président
--
rencontre
--
entre
--
syndicat
--
martelle
--
normal
--
regarder
----------------
word: positif
closest words: 
--
positif
--
chose
--
cousin
--
ensuite
--
partie
--
montélimar
--
faire
----------------
word: hydroxychloroquine
closest words: 
--
hydroxychloroquine
--
raoult
--
défense
--
évoquer
--
pgqmebj
--
mobiliser
--

Epoch 16/30
108/110 [============================>.] - ETA: 0s - loss: 0.0547word: président
closest words: 
--
président
--
entre
--
rencontre
--
chancelier
--
martelle
--
syndicat
--
httpstcogl
----------------
word: positif
closest words: 
--
positif
--
ensuite
--
cousin
--
partie
--
chose
--
beau
--
montélimar
----------------
word: hydroxychloroquine
closest words: 
--
hydroxychloroquine
--
raoult
--
défense
--
évoquer
--
remettre
--
moitié
--
mobiliser
----------------
110/110 [==============================] - 0s 4ms/step - loss: 0.0547
Epoch 17/30
 98/110 [=========================>....] - ETA: 0s - loss: 0.0399word: président
closest words: 
--
président
--
entre
--
rencontre
--
chancelier
--
martelle
--
httpstcogl
--
syndicat
----------------
word: positif
closest words: 
--
positif
--
ensuite
--
cousin
--
partie
--
chose
--
beau
--
montélimar
----------------
word: hydroxychloroquine
closest words: 
--
hydroxychloroquine
--
raoult
--
défense
--
évoquer
--
remettre
--
moitié


#### Code for stemming

In [32]:
## https://stackoverflow.com/questions/13131139/lemmatize-french-text

# from nltk.stem.snowball import FrenchStemmer

# stemmer = FrenchStemmer()

# def stem(tokens):
#     return [stemmer.stem(x) for x in tokens]

# df['Title'] = df['Title'].map(
#     stem
# )